# Estimating local and global feature importance scores using DiCE

Summaries of counterfactual examples can be used to estimate importance of features. Intuitively, a feature that is changed more often to generate a proximal counterfactual is an important feature. We use this intuition to build a feature importance score. 

This score can be interpreted as a measure of the **necessity** of a feature to cause a particular model output. That is, if the feature's value changes, then it is likely that the model's output class will also change (or the model's output will significantly change in case of regression model).  

Below we show how counterfactuals can be used to provide local feature importance scores for any input, and how those scores can be combined to yield a global importance score for each feature.

In [1]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

import dice_ml
from dice_ml import Dice
from dice_ml.utils import helpers  # helper functions

In [2]:
%load_ext autoreload
%autoreload 2

## Preliminaries: Loading the data and ML model

In [3]:
dataset = helpers.load_adult_income_dataset().sample(5000)  # downsampling to reduce ML model fitting time
helpers.get_adult_data_info()

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

In [4]:
target = dataset["income"]

# Split data into train and test
datasetX = dataset.drop("income", axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=0,
                                                    stratify=target)

numerical = ["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical)

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])
model = clf.fit(x_train, y_train)

In [5]:
d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')
m = dice_ml.Model(model=model, backend="sklearn")

## Local feature importance

We first generate counterfactuals for a given input point. 

In [6]:
exp = Dice(d, m, method="random")
query_instance = x_train[1:2]
e1 = exp.generate_counterfactuals(query_instance, total_CFs=10, desired_range=None,
                                  desired_class="opposite",
                                  permitted_range=None, features_to_vary="all")
e1.visualize_as_dataframe(show_only_changes=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.96s/it]

Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,41,Private,Bachelors,Married,Sales,White,Male,48,1



Diverse Counterfactual set (new outcome: 0.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,19.0,-,-,-,-,Other,-,-,0
1,-,-,-,-,-,Other,-,9.0,0
2,-,-,-,Single,-,Other,-,-,0
3,-,-,School,-,White-Collar,-,-,-,0
4,-,-,-,-,Blue-Collar,-,Female,-,0
5,-,-,-,-,Blue-Collar,-,-,21.0,0
6,-,-,-,Divorced,Blue-Collar,-,-,-,0
7,24.0,-,-,-,Blue-Collar,-,-,-,0
8,-,-,Assoc,Separated,-,-,-,-,0
9,-,-,-,-,-,-,-,22.0,0


These can now be used to calculate the feature importance scores. 

In [7]:
imp = exp.local_feature_importance(query_instance, cf_examples_list=e1.cf_examples_list)
print(imp.local_importance)

[{'occupation': 0.5, 'marital_status': 0.3, 'race': 0.3, 'hours_per_week': 0.3, 'education': 0.2, 'age': 0.2, 'gender': 0.1, 'workclass': 0.0}]


Feature importance can also be estimated directly, by leaving the `cf_examples_list` argument blank.

In [8]:
imp = exp.local_feature_importance(query_instance, posthoc_sparsity_param=None)
print(imp.local_importance)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.66it/s]

[{'age': 0.7, 'hours_per_week': 0.6, 'occupation': 0.2, 'workclass': 0.1, 'marital_status': 0.1, 'race': 0.1, 'education': 0.0, 'gender': 0.0}]


## Global importance

For global importance, we need to generate counterfactuals for a representative sample of the dataset. 

In [9]:
cobj = exp.global_feature_importance(x_train[0:10], total_CFs=10, posthoc_sparsity_param=None)
print(cobj.summary_importance)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.80it/s]

{'age': 0.5, 'hours_per_week': 0.38, 'education': 0.27, 'marital_status': 0.27, 'occupation': 0.19, 'race': 0.11, 'workclass': 0.08, 'gender': 0.05}


## Convert the counterfactual output to json

In [10]:
json_str = cobj.to_json()
print(json_str)

{"test_data": [[[36, "Private", "Assoc", "Married", "Blue-Collar", "White", "Male", 38, 0]], [[41, "Private", "Bachelors", "Married", "Sales", "White", "Male", 48, 1]], [[32, "Government", "Some-college", "Single", "White-Collar", "White", "Male", 40, 0]], [[35, "Private", "Masters", "Married", "Professional", "White", "Male", 38, 0]], [[32, "Private", "Bachelors", "Married", "White-Collar", "White", "Male", 50, 1]], [[33, "Government", "Some-college", "Married", "Service", "White", "Male", 45, 0]], [[31, "Private", "Some-college", "Single", "White-Collar", "White", "Female", 43, 0]], [[43, "Private", "Assoc", "Married", "Blue-Collar", "White", "Male", 40, 0]], [[42, "Government", "Assoc", "Divorced", "Professional", "Other", "Female", 40, 1]], [[43, "Private", "Masters", "Married", "White-Collar", "White", "Male", 45, 1]]], "cfs_list": [[[36, "Private", "Assoc", "Married", "Other/Unknown", "White", "Male", 14.0, 1], [36, "Private", "Assoc", "Married", "Blue-Collar", "White", "Male", 9

## Convert the json output to a counterfactual object

In [11]:
imp_r = imp.from_json(json_str)
print([o.visualize_as_dataframe(show_only_changes=True) for o in imp_r.cf_examples_list])
print(imp_r.local_importance)
print(imp_r.summary_importance)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,36,Private,Assoc,Married,Blue-Collar,White,Male,38,0



Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,-,-,Other/Unknown,-,-,14.0,1
1,-,-,-,-,-,-,-,97.0,-
2,-,-,-,-,Professional,-,-,13.0,1
3,88.0,-,-,-,-,-,-,-,1
4,-,-,-,-,Sales,Other,-,-,1
5,-,-,Some-college,-,-,-,-,66.0,1
6,-,-,-,-,-,-,-,44.0,1
7,61.0,-,-,-,Sales,-,-,-,1
8,86.0,-,-,-,-,-,-,-,1
9,56.0,-,-,-,-,-,-,-,1


Query instance (original outcome : 1)


...

Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,43,Private,Masters,Married,White-Collar,White,Male,45,1



Counterfactual set (new outcome: 0.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,School,Single,-,-,-,-,0
1,-,-,HS-grad,-,-,-,-,3.0,0
2,-,-,-,Widowed,-,Other,-,-,0
3,67.0,-,-,Divorced,-,-,-,-,0
4,18.0,-,-,-,-,-,-,92.0,0
5,70.0,-,-,Widowed,-,-,-,-,0
6,80.0,-,-,-,-,-,-,37.0,0
7,81.0,-,School,-,-,-,-,-,0
8,42.0,-,-,Divorced,-,-,-,-,0
9,24.0,-,Assoc,-,-,-,-,-,0


[None, None, None, None, None, None, None, None, None, None]
[{'hours_per_week': 0.5, 'age': 0.4, 'occupation': 0.4, 'education': 0.1, 'race': 0.1, 'workclass': 0.0, 'marital_status': 0.0, 'gender': 0.0}, {'age': 0.5, 'hours_per_week': 0.5, 'education': 0.4, 'workclass': 0.2, 'gender': 0.2, 'marital_status': 0.1, 'occupation': 0.0, 'race': 0.0}, {'marital_status': 0.8, 'age': 0.6, 'education': 0.3, 'hours_per_week': 0.2, 'workclass': 0.0, 'occupation': 0.0, 'race': 0.0, 'gender': 0.0}, {'hours_per_week': 0.5, 'occupation': 0.4, 'age': 0.3, 'race': 0.2, 'education': 0.1, 'marital_status': 0.1, 'gender': 0.1, 'workclass': 0.0}, {'age': 0.5, 'hours_per_week': 0.5, 'education': 0.2, 'workclass': 0.1, 'occupation': 0.1, 'race': 0.1, 'marital_status': 0.0, 'gender': 0.0}, {'age': 0.7, 'education': 0.5, 'occupation': 0.2, 'race': 0.2, 'hours_per_week': 0.2, 'gender': 0.1, 'workclass': 0.0, 'marital_status': 0.0}, {'marital_status': 0.7, 'education': 0.6, 'age': 0.4, 'occupation': 0.2, 'hours_